In [68]:
# Useful starting lines
%matplotlib inline
import numpy as np
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Read data

In [69]:
from sqlalchemy import create_engine
import pandas as pd

path = "../Data/wjazzd.db"
engine = create_engine(f"sqlite:///{path}")

melody = pd.read_sql("melody", engine)
beats = pd.read_sql("beats", engine)

df_melody = pd.DataFrame(melody)
df_beats = pd.DataFrame(beats)

## Fill the chords column with the last known chord

In [70]:
#df_chords = df_beats[df_beats['chord'] != '']
df_chords = df_beats.replace({'chord': {'': np.nan}}).ffill()
#df_chords.to_csv('df_chords.csv')
df_chords = df_beats.copy()
df_chords.info()
df_chords.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132329 entries, 0 to 132328
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   beatid      132329 non-null  int64  
 1   melid       132329 non-null  int64  
 2   onset       132329 non-null  float64
 3   bar         132329 non-null  int64  
 4   beat        132329 non-null  int64  
 5   signature   132329 non-null  object 
 6   chord       132329 non-null  object 
 7   form        132329 non-null  object 
 8   bass_pitch  122540 non-null  float64
 9   chorus_id   132329 non-null  int64  
dtypes: float64(2), int64(5), object(3)
memory usage: 10.1+ MB


,beatid,melid,onset,bar,beat,signature,chord,form,bass_pitch,chorus_id
0,1,1,9.171882,-1,1,,,I1,42.0,0
1,2,1,9.488254,-1,2,,,,42.0,0
2,3,1,9.779955,-1,3,,,,40.0,0
3,4,1,10.052608,-1,4,,,,40.0,0
4,5,1,10.339796,0,1,,Bb6,,50.0,0
5,6,1,10.631542,0,2,,,,50.0,0
6,7,1,10.918163,0,3,,,,50.0,0
7,8,1,11.217007,0,4,,,,42.0,0
8,9,1,11.514127,1,1,4/4,Bb6,A1,42.0,1
9,10,1,11.824150,1,2,,,,42.0,1


## Extract corresponding row in beats using the (melid, bar, beat) features

In [71]:
# Define new index with the key (melid, bar, beat)
new_index = ['melid', 'bar', 'beat']
df_chords_new = df_chords.set_index(new_index, drop=True)
df_melody_new = df_melody.set_index(new_index, drop=True)

# Merge the dataframes using the new index and then reset the index again
df_beats_mel = df_chords_new.merge(df_melody_new, left_on=new_index, right_on=new_index, how='outer')
df_beats_mel = df_beats_mel.reset_index(drop=False)

df_beats_mel = df_beats_mel[['melid', 'bar', 'beat', 'beatid', 'onset_x', 'chord', 'bass_pitch', 'eventid', 'pitch', 'duration']]

df_beats_mel.head(15)


,melid,bar,beat,beatid,onset_x,chord,bass_pitch,eventid,pitch,duration
0,1,-1,1,1,9.171882,,42.0,NaN,NaN,NaN
1,1,-1,2,2,9.488254,,42.0,NaN,NaN,NaN
2,1,-1,3,3,9.779955,,40.0,NaN,NaN,NaN
3,1,-1,4,4,10.052608,,40.0,NaN,NaN,NaN
4,1,0,1,5,10.339796,Bb6,50.0,1.0,65.0,0.138776
5,1,0,2,6,10.631542,,50.0,2.0,63.0,0.171247
6,1,0,2,6,10.631542,,50.0,3.0,58.0,0.081270
7,1,0,3,7,10.918163,,50.0,4.0,61.0,0.235102
8,1,0,4,8,11.217007,,42.0,5.0,63.0,0.130612
9,1,1,1,9,11.514127,Bb6,42.0,6.0,58.0,0.188662


In [72]:
df_beats_mel.loc[df_beats_mel['melid']==2].head(15)

,melid,bar,beat,beatid,onset_x,chord,bass_pitch,eventid,pitch,duration
735,2,1,1,530,2.780590,Eb7,34.0,NaN,NaN,NaN
736,2,1,2,531,3.105669,,36.0,531.0,70.0,0.119002
737,2,1,3,532,3.413333,,37.0,532.0,73.0,0.156735
738,2,1,3,532,3.413333,,37.0,533.0,75.0,0.110295
739,2,1,4,533,3.725420,,38.0,NaN,NaN,NaN
740,2,2,1,534,4.032653,Ab7,39.0,534.0,63.0,0.081270
741,2,2,2,535,4.351020,,40.0,535.0,66.0,0.148027
742,2,2,3,536,4.669388,,41.0,536.0,68.0,0.557279
743,2,2,4,537,4.991610,,41.0,NaN,NaN,NaN
744,2,3,1,538,5.297959,Eb7,42.0,NaN,NaN,NaN


In [73]:
df_beats_mel.loc[df_beats_mel['melid']==3].head(15)

,melid,bar,beat,beatid,onset_x,chord,bass_pitch,eventid,pitch,duration
1590,3,0,1,1061,2.052063,C7alt,48.0,1197.0,64.0,0.103039
1591,3,0,1,1061,2.052063,C7alt,48.0,1198.0,60.0,0.062404
1592,3,0,2,1062,2.487438,,36.0,1199.0,58.0,0.111746
1593,3,0,2,1062,2.487438,,36.0,1200.0,62.0,0.127710
1594,3,0,3,1063,2.893878,,37.0,1201.0,60.0,0.119002
1595,3,0,3,1063,2.893878,,37.0,1202.0,58.0,0.091429
1596,3,0,3,1063,2.893878,,37.0,1203.0,61.0,0.095782
1597,3,0,4,1064,3.330612,,36.0,1204.0,63.0,0.111746
1598,3,0,4,1064,3.330612,,36.0,1205.0,61.0,0.081633
1599,3,0,4,1064,3.330612,,36.0,1206.0,60.0,0.077551


In [74]:
import sys
sys.path.append('../data')

from combine_melody_beats import encode_melody
from multi_hot_encoding import preprocess_chords, encode_chords_1

## Note sequence encoding

In [75]:
#beats_encoded.drop(['signature', 'onset', 'form', 'chorus_id'], axis=1, inplace=True)
pitch_per_chord = encode_melody(df_melody, df_beats, pitch_sequence=False)
pitch_sequence_per_chord = encode_melody(df_melody, df_beats, pitch_sequence=True)
pitch_per_chord = preprocess_chords(pitch_per_chord)
pitch_sequence_per_chord = preprocess_chords(pitch_sequence_per_chord)


../data\multi_hot_encoding.py:86: FutureWarning: The default value of regex will change from True to False in a future version.
  beats['chord_map'] = beats['chord_map'].str.replace('\/(.*)','')
../data\multi_hot_encoding.py:130: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  beats['added_note'] = beats['added_note'].str.replace(i,'')
../data\multi_hot_encoding.py:86: FutureWarning: The default value of regex will change from True to False in a future version.
  beats['chord_map'] = beats['chord_map'].str.replace('\/(.*)','')
../data\multi_hot_encoding.py:130: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  beats['added_note'] = beats['added_note'].str.replace(i,'')


In [76]:
pitch_per_chord.head(15)

,melid,bar,beat,beatid,onset,signature,chord,form,bass_pitch,chorus_id,...,pitch,duration,pitch_encoded,bass_pitch_encoded,chord_info,Root_pitch,chord_map,new_chord,triad,added_note
0,1,0,1,5.0,10.339796,-1,Bb6,-1,50.0,0.0,...,65.0,0.1388,5,2,6,A#,6,A#6,M,6
1,1,0,2,6.0,10.631542,-1,Bb6,-1,50.0,0.0,...,63.0,0.1712,3,2,6,A#,6,A#6,M,6
2,1,0,2,6.0,10.631542,-1,Bb6,-1,50.0,0.0,...,58.0,0.0813,10,2,6,A#,6,A#6,M,6
3,1,0,3,7.0,10.918163,-1,Bb6,-1,50.0,0.0,...,61.0,0.2351,1,2,6,A#,6,A#6,M,6
4,1,0,4,8.0,11.217007,-1,Bb6,-1,42.0,0.0,...,63.0,0.1306,3,6,6,A#,6,A#6,M,6
5,1,1,1,9.0,11.514127,4/4,Bb6,A1,42.0,1.0,...,58.0,0.1887,10,6,6,A#,6,A#6,M,6
6,1,1,2,10.0,11.824150,-1,Bb6,-1,42.0,1.0,...,58.0,0.4818,10,6,6,A#,6,A#6,M,6
7,1,1,3,11.0,12.129433,-1,G7,-1,43.0,1.0,...,-1.0,-1.0000,-1,7,7,G,7,G7,M,m7
8,1,1,4,12.0,12.414785,-1,G7,-1,43.0,1.0,...,-1.0,-1.0000,-1,7,7,G,7,G7,M,m7
9,1,2,1,13.0,12.703628,-1,C-7,-1,39.0,1.0,...,-1.0,-1.0000,-1,3,-7,C,-7,C-7,-,m7


In [77]:
pitch_sequence_per_chord.head(10)

,melid,bar,beat,beatid,onset,signature,chord,form,bass_pitch,chorus_id,...,pitch_encoded,pitch_sequence,bass_pitch_sequence,duration_sequence,chord_info,Root_pitch,chord_map,new_chord,triad,added_note
6,1,1,2,10.0,11.824150,-1,Bb6,-1,42.0,1.0,...,10,"[5, 3, 10, 1, 3, 10, 10]","[2, 2, 2, 2, 6, 6, 6]","[0.1388, 0.1712, 0.0813, 0.2351, 0.1306, 0.188...",6,A#,6,A#6,M,6
8,1,1,4,12.0,12.414785,-1,G7,-1,43.0,1.0,...,-1,"[-1, -1]","[7, 7]","[-1.0, -1.0]",7,G,7,G7,M,m7
10,1,2,2,14.0,12.983560,-1,C-7,-1,40.0,1.0,...,-1,"[-1, -1]","[3, 4]","[-1.0, -1.0]",-7,C,-7,C-7,-,m7
12,1,2,4,16.0,13.608299,-1,F7,-1,42.0,1.0,...,-1,"[-1, -1]","[5, 6]","[-1.0, -1.0]",7,F,7,F7,M,m7
14,1,3,2,18.0,14.192585,-1,Bb,-1,39.0,1.0,...,-1,"[-1, -1]","[2, 3]","[-1.0, -1.0]",,A#,,A#,M,none
17,1,3,4,20.0,14.747211,-1,G-7,-1,42.0,1.0,...,0,"[2, 9, 0]","[5, 6, 6]","[0.1596, 0.1451, 0.1103]",-7,G,-7,G-7,-,m7
20,1,4,2,22.0,15.359501,-1,C-7,-1,38.0,1.0,...,10,"[10, 7, 10]","[7, 7, 2]","[0.1306, 0.0972, 0.09]",-7,C,-7,C-7,-,m7
23,1,4,4,24.0,15.943197,-1,F7,-1,33.0,1.0,...,10,"[1, 0, 10]","[3, 9, 9]","[0.283, 0.1509, 0.0871]",7,F,7,F7,M,m7
25,1,5,2,26.0,16.516508,-1,F-7,-1,32.0,1.0,...,10,"[0, 10]","[10, 8]","[0.3837, 0.1335]",-7,F,-7,F-7,-,m7
29,1,5,4,28.0,17.089138,-1,Bb7,-1,32.0,1.0,...,9,"[8, 0, 11, 9]","[6, 6, 8, 8]","[0.1364, 0.1263, 0.1059, 0.1001]",7,A#,7,A#7,M,m7


In [78]:
pitch_sequence_per_chord.loc[pitch_sequence_per_chord['melid']==2].head(10)

,melid,bar,beat,beatid,onset,signature,chord,form,bass_pitch,chorus_id,...,pitch_encoded,pitch_sequence,bass_pitch_sequence,duration_sequence,chord_info,Root_pitch,chord_map,new_chord,triad,added_note
735,2,1,4,533.0,3.725420,-1,Eb7,-1,38.0,1.0,...,-1,"[-1, 10, 1, 3, -1]","[10, 0, 1, 1, 2]","[-1.0, 0.119, 0.1567, 0.1103, -1.0]",7,D#,7,D#7,M,m7
739,2,2,4,537.0,4.991610,-1,Ab7,-1,41.0,1.0,...,-1,"[3, 6, 8, -1]","[3, 4, 5, 5]","[0.0813, 0.148, 0.5573, -1.0]",7,G#,7,G#7,M,m7
748,2,4,4,545.0,7.538685,-1,Eb7,-1,37.0,1.0,...,10,"[-1, 3, 7, 10, -1, -1, -1, 7, 10]","[6, 7, 8, 8, 2, 1, 1, 1, 1]","[-1.0, 0.0842, 0.1771, 0.1016, -1.0, -1.0, -1....",7,D#,7,D#7,M,m7
758,2,6,4,553.0,10.026327,-1,Ab7,-1,43.0,1.0,...,1,"[-1, 10, 3, 10, -1, 10, 1, 3, -1, 1]","[0, 0, 0, 0, 0, 10, 10, 10, 9, 7]","[-1.0, 0.1103, 0.1887, 0.0958, -1.0, 0.0871, 0...",7,G#,7,G#7,M,m7
768,2,8,4,561.0,12.499728,-1,Eb7,-1,37.0,1.0,...,1,"[5, 10, -1, -1, -1, -1, 10, 3, 10, 1]","[7, 7, 8, 10, 11, 0, 4, 4, 7, 1]","[0.1887, 0.1422, -1.0, -1.0, -1.0, -1.0, 0.095...",7,D#,7,D#7,M,m7
774,2,9,4,565.0,13.730544,-1,F-7,-1,39.0,1.0,...,8,"[-1, 10, 3, 1, 10, 8]","[0, 7, 5, 5, 3, 3]","[-1.0, 0.1248, 0.1654, 0.1248, 0.1712, 0.6444]",-7,F,-7,F-7,-,m7
778,2,10,4,569.0,14.967347,-1,Bb7,-1,43.0,1.0,...,8,"[-1, -1, 10, 8]","[2, 0, 10, 7]","[-1.0, -1.0, 0.4702, 0.1596]",7,A#,7,A#7,M,m7
784,2,11,4,573.0,16.233424,-1,Eb7,-1,43.0,1.0,...,-1,"[5, 7, 3, 1, 3, -1]","[7, 7, 11, 11, 0, 7]","[0.0784, 0.2322, 0.2032, 0.0668, 0.5312, -1.0]",7,D#,7,D#7,M,m7
788,2,12,4,577.0,17.499274,-1,Bb7,-1,43.0,1.0,...,-1,"[-1, -1, -1, -1]","[5, 7, 8, 7]","[-1.0, -1.0, -1.0, -1.0]",7,A#,7,A#7,M,m7
795,2,13,4,581.0,18.736531,-1,Eb7,-1,43.0,2.0,...,3,"[3, 3, 3, 7, 10, 0, 3]","[3, 3, 3, 8, 8, 7, 7]","[0.1132, 0.1132, 0.1248, 0.1829, 0.0493, 0.200...",7,D#,7,D#7,M,m7


In [79]:
pitch_sequence_per_chord.loc[pitch_sequence_per_chord['melid']==3].head(10)

,melid,bar,beat,beatid,onset,signature,chord,form,bass_pitch,chorus_id,...,pitch_encoded,pitch_sequence,bass_pitch_sequence,duration_sequence,chord_info,Root_pitch,chord_map,new_chord,triad,added_note
1596,3,0,4,1064.0,3.330612,-1,C7alt,-1,36.0,0.0,...,10,"[4, 0, 10, 2, 0, 10, 1, 3, 1, 0, 10]","[0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0]","[0.103, 0.0624, 0.1117, 0.1277, 0.119, 0.0914,...",7alt,C,7,C7,M,m7
1612,3,2,4,1072.0,6.862177,-1,Fj7,-1,38.0,1.0,...,0,"[9, -1, 0, 9, 0, 4, 7, 4, 0, 4, 7, 4, 5, 9, 2, 0]","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 9, 9, 9, 2, 2]","[0.1194, -1.0, 0.1088, 0.0551, 0.0856, 0.1408,...",j7,F,j7,Fj7,M,j7
1623,3,4,4,1080.0,10.287664,-1,G7alt,-1,43.0,1.0,...,7,"[-1, -1, 7, 9, 11, 3, 11, 2, 11, 2, 7]","[7, 2, 11, 0, 0, 2, 7, 7, 7, 7, 7]","[-1.0, -1.0, 0.0929, 0.1045, 0.1265, 0.3306, 0...",7alt,G,7,G7,M,m7
1630,3,5,4,1084.0,12.004218,-1,G-7,-1,43.0,1.0,...,4,"[-1, -1, 10, 9, 7, 5, 4]","[7, 7, 1, 1, 7, 7, 7]","[-1.0, -1.0, 0.0806, 0.101, 0.102, 0.0765, 0.1...",-7,G,-7,G-7,-,m7
1639,3,6,4,1088.0,13.700068,-1,C7,-1,34.0,1.0,...,0,"[7, 5, 4, 2, 0, 0, 2, 7, 0]","[0, 0, 0, 0, 10, 10, 10, 10, 10]","[0.101, 0.1061, 0.0796, 0.098, 0.0726, 0.0856,...",7,C,7,C7,M,m7
1644,3,7,4,1092.0,15.395850,-1,Am7b5,-1,33.0,1.0,...,-1,"[-1, -1, 9, 0, -1]","[9, 9, 9, 9, 9]","[-1.0, -1.0, 0.0595, 0.1524, -1.0]",m7b5,A,-7b5,A-7b5,half,m7
1656,3,8,4,1096.0,17.042245,-1,D79b,-1,38.0,1.0,...,9,"[-1, 3, 2, 0, 10, 8, 9, 0, 9, 10, 7, 9]","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]","[-1.0, 0.0856, 0.0798, 0.1001, 0.0755, 0.0689,...",79b,D,7,D7,M,m7
1661,3,9,4,1100.0,18.763175,-1,G-7,-1,38.0,1.0,...,-1,"[-1, 9, 0, 10, -1]","[10, 10, 2, 2, 2]","[-1.0, 0.1245, 0.2204, 0.1694, -1.0]",-7,G,-7,G-7,-,m7
1668,3,10,4,1104.0,20.464580,-1,A79b,-1,40.0,1.0,...,4,"[-1, 9, 7, 10, 9, 3, 4]","[9, 9, 9, 4, 4, 4, 4]","[-1.0, 0.2327, 0.1, 0.2408, 0.1571, 0.1878, 0....",79b,A,7,A7,M,m7
1676,3,11,4,1108.0,22.177528,-1,D7,-1,38.0,1.0,...,2,"[6, 9, 6, -1, 2, 4, 1, 2]","[2, 2, 2, 9, 2, 2, 2, 2]","[0.3796, 0.2408, 0.2163, -1.0, 0.0612, 0.1184,...",7,D,7,D7,M,m7


## Create tuple with pitch and duration

In [80]:
beats = beats[['beatid', 'melid', 'bar', 'beat', 'chord', 'bass_pitch']]

beats_mel = encode_melody(df_melody, df_beats, pitch_sequence=True)
beats_mel = preprocess_chords(beats_mel)    
beats_mel = encode_chords_1(beats_mel)
pitch_per_chord = encode_melody(df_melody, df_beats, pitch_sequence=False)


beats_mel.head(10)

../data\multi_hot_encoding.py:86: FutureWarning: The default value of regex will change from True to False in a future version.
  beats['chord_map'] = beats['chord_map'].str.replace('\/(.*)','')
../data\multi_hot_encoding.py:130: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  beats['added_note'] = beats['added_note'].str.replace(i,'')


,melid,bar,beat,beatid,onset,signature,chord,form,bass_pitch,chorus_id,...,chord_info,Root_pitch,chord_map,new_chord,triad,added_note,Root_pitch_num,triad_num,added_note_num,new_chord_num
6,1,1,2,10.0,11.824150,-1,Bb6,-1,42.0,1.0,...,6,A#,6,A#6,M,6,0,0,0,0
8,1,1,4,12.0,12.414785,-1,G7,-1,43.0,1.0,...,7,G,7,G7,M,m7,1,0,1,1
10,1,2,2,14.0,12.983560,-1,C-7,-1,40.0,1.0,...,-7,C,-7,C-7,-,m7,2,1,1,2
12,1,2,4,16.0,13.608299,-1,F7,-1,42.0,1.0,...,7,F,7,F7,M,m7,3,0,1,3
14,1,3,2,18.0,14.192585,-1,Bb,-1,39.0,1.0,...,,A#,,A#,M,none,0,0,2,4
17,1,3,4,20.0,14.747211,-1,G-7,-1,42.0,1.0,...,-7,G,-7,G-7,-,m7,1,1,1,5
20,1,4,2,22.0,15.359501,-1,C-7,-1,38.0,1.0,...,-7,C,-7,C-7,-,m7,2,1,1,2
23,1,4,4,24.0,15.943197,-1,F7,-1,33.0,1.0,...,7,F,7,F7,M,m7,3,0,1,3
25,1,5,2,26.0,16.516508,-1,F-7,-1,32.0,1.0,...,-7,F,-7,F-7,-,m7,3,1,1,6
29,1,5,4,28.0,17.089138,-1,Bb7,-1,32.0,1.0,...,7,A#,7,A#7,M,m7,0,0,1,7


In [81]:
df_beats.loc[df_beats['melid']==1].head(20)

,beatid,melid,onset,bar,beat,signature,chord,form,bass_pitch,chorus_id
0,1,1,9.171882,-1,1,,,I1,42.0,0
1,2,1,9.488254,-1,2,,,,42.0,0
2,3,1,9.779955,-1,3,,,,40.0,0
3,4,1,10.052608,-1,4,,,,40.0,0
4,5,1,10.339796,0,1,,Bb6,,50.0,0
5,6,1,10.631542,0,2,,,,50.0,0
6,7,1,10.918163,0,3,,,,50.0,0
7,8,1,11.217007,0,4,,,,42.0,0
8,9,1,11.514127,1,1,4/4,Bb6,A1,42.0,1
9,10,1,11.824150,1,2,,,,42.0,1


In [82]:
pitch_per_chord[['melid','bar', 'beat', 'beatid', 'pitch', 'pitch_encoded']].loc[pitch_per_chord['melid']==1].head(20)

,melid,bar,beat,beatid,pitch,pitch_encoded
0,1,0,1,5.0,65.0,5
1,1,0,2,6.0,63.0,3
2,1,0,2,6.0,58.0,10
3,1,0,3,7.0,61.0,1
4,1,0,4,8.0,63.0,3
5,1,1,1,9.0,58.0,10
6,1,1,2,10.0,58.0,10
7,1,1,3,11.0,-1.0,-1
8,1,1,4,12.0,-1.0,-1
9,1,2,1,13.0,-1.0,-1


In [83]:
beats_mel[['melid','bar', 'beat', 'beatid', 'pitch_sequence']].loc[beats_mel['melid']==1].head(10)

,melid,bar,beat,beatid,pitch_sequence
6,1,1,2,10.0,"[5, 3, 10, 1, 3, 10, 10]"
8,1,1,4,12.0,"[-1, -1]"
10,1,2,2,14.0,"[-1, -1]"
12,1,2,4,16.0,"[-1, -1]"
14,1,3,2,18.0,"[-1, -1]"
17,1,3,4,20.0,"[2, 9, 0]"
20,1,4,2,22.0,"[10, 7, 10]"
23,1,4,4,24.0,"[1, 0, 10]"
25,1,5,2,26.0,"[0, 10]"
29,1,5,4,28.0,"[8, 0, 11, 9]"


In [84]:
df_beats.loc[df_beats['melid']==4].head(10)

,beatid,melid,onset,bar,beat,signature,chord,form,bass_pitch,chorus_id
1324,1325,4,2.600726,-1,1,,Eb6,I1,39.0,0
1325,1326,4,3.018367,-1,2,,,,39.0,0
1326,1327,4,3.422449,-1,3,,,,39.0,0
1327,1328,4,3.797098,-1,4,,,,39.0,0
1328,1329,4,4.200000,0,1,,,,39.0,0
1329,1330,4,4.575760,0,2,,,,39.0,0
1330,1331,4,4.957846,0,3,,,,39.0,0
1331,1332,4,5.337528,0,4,,,,28.0,0
1332,1333,4,5.717279,1,1,4/4,F7,A1,41.0,1
1333,1334,4,6.107166,1,2,,,,41.0,1


In [85]:
beats_mel.loc[beats_mel['melid']==4].head(10)

,melid,bar,beat,beatid,onset,signature,chord,form,bass_pitch,chorus_id,...,chord_info,Root_pitch,chord_map,new_chord,triad,added_note,Root_pitch_num,triad_num,added_note_num,new_chord_num
2060,4,0,4,1332.0,5.337528,-1,Eb6,-1,28.0,0.0,...,6,D#,6,D#6,M,6,4,0,0,27
2066,4,1,4,1336.0,6.888277,-1,F7,-1,41.0,1.0,...,7,F,7,F7,M,m7,3,0,1,3
2074,4,2,4,1340.0,8.437596,-1,Bb7,-1,28.0,1.0,...,7,A#,7,A#7,M,m7,0,0,1,7
2085,4,4,4,1348.0,11.471542,-1,Eb6,-1,42.0,1.0,...,6,D#,6,D#6,M,6,4,0,0,27
2092,4,5,4,1352.0,12.997891,-1,Bb-7,-1,36.0,1.0,...,-7,A#,-7,A#-7,-,m7,0,1,1,28
2099,4,6,4,1356.0,14.500862,-1,Eb7,-1,43.0,1.0,...,7,D#,7,D#7,M,m7,4,0,1,8
2114,4,8,4,1364.0,17.466644,-1,Abj7,-1,41.0,1.0,...,j7,G#,j7,G#j7,M,j7,5,0,3,29
2120,4,9,4,1368.0,18.971973,-1,Ab6,-1,34.0,1.0,...,6,G#,6,G#6,M,6,5,0,0,30
2124,4,10,4,1372.0,20.465306,-1,Db7,-1,34.0,1.0,...,7,C#,7,C#7,M,m7,11,0,1,31
2130,4,11,4,1376.0,21.945601,-1,Eb6,-1,38.0,1.0,...,6,D#,6,D#6,M,6,4,0,0,27


In [86]:
df_beats.loc[df_beats['melid']==5].head(10)

,beatid,melid,onset,bar,beat,signature,chord,form,bass_pitch,chorus_id
1840,1841,5,5.131973,0,4,,NC,I1,39.0,0
1841,1842,5,6.153288,1,1,4/4,Ab6,A1,44.0,1
1842,1843,5,7.183673,1,2,,,,43.0,1
1843,1844,5,8.241678,1,3,,,,41.0,1
1844,1845,5,9.209841,1,4,,,,40.0,1
1845,1846,5,10.195918,2,1,,Ab6,,39.0,1
1846,1847,5,11.155805,2,2,,,,37.0,1
1847,1848,5,12.085986,2,3,,,,36.0,1
1848,1849,5,13.028571,2,4,,,,33.0,1
1849,1850,5,13.975510,3,1,,Ab-6,,38.0,1


In [87]:
beats_mel.loc[beats_mel['melid']==5].head(10)

,melid,bar,beat,beatid,onset,signature,chord,form,bass_pitch,chorus_id,...,chord_info,Root_pitch,chord_map,new_chord,triad,added_note,Root_pitch_num,triad_num,added_note_num,new_chord_num
2803,5,2,4,1849.0,13.028571,-1,Ab6,-1,33.0,1.0,...,6,G#,6,G#6,M,6,5,0,0,30
2813,5,4,2,1855.0,18.700091,-1,Ab-6,-1,35.0,1.0,...,-6,G#,-6,G#-6,-,6,5,1,0,34
2817,5,4,4,1857.0,20.670363,-1,Db7,-1,34.0,1.0,...,7,C#,7,C#7,M,m7,11,0,1,31
2823,5,5,2,1859.0,22.636032,-1,Eb,-1,39.0,1.0,...,,D#,,D#,M,none,4,0,2,35
2825,5,5,4,1861.0,24.546508,-1,F-7,-1,38.0,1.0,...,-7,F,-7,F-7,-,m7,3,1,1,6
2828,5,6,2,1863.0,26.547166,-1,G-7,-1,43.0,1.0,...,-7,G,-7,G-7,-,m7,1,1,1,5
2832,5,6,4,1865.0,28.528118,-1,C7,-1,40.0,1.0,...,7,C,7,C7,M,m7,2,0,1,12
2838,5,7,3,1868.0,31.515488,-1,F-7,-1,31.0,1.0,...,-7,F,-7,F-7,-,m7,3,1,1,6
2841,5,7,4,1869.0,32.483719,-1,Eo7,-1,36.0,1.0,...,o7,E,o7,Eo7,o,o7,10,3,4,36
2848,5,8,4,1873.0,36.409955,-1,F-7,-1,43.0,1.0,...,-7,F,-7,F-7,-,m7,3,1,1,6


In [88]:
df_beats.loc[df_beats['melid']==10].head(13)

,beatid,melid,onset,bar,beat,signature,chord,form,bass_pitch,chorus_id
2921,2922,10,5.151134,0,1,,NC,I1,36.0,0
2922,2923,10,5.434354,0,2,,,,28.0,0
2923,2924,10,5.722449,0,3,,,,41.0,0
2924,2925,10,6.019637,0,4,,,,45.0,0
2925,2926,10,6.297778,1,1,4/4,Bbj7,A1,34.0,1
2926,2927,10,6.582857,1,2,,,,38.0,1
2927,2928,10,6.855692,1,3,,,,43.0,1
2928,2929,10,7.145941,1,4,,,,42.0,1
2929,2930,10,7.433288,2,1,,,,41.0,1
2930,2931,10,7.720408,2,2,,,,33.0,1


In [89]:
beats_mel.loc[beats_mel['melid']==10].head(10)

,melid,bar,beat,beatid,onset,signature,chord,form,bass_pitch,chorus_id,...,chord_info,Root_pitch,chord_map,new_chord,triad,added_note,Root_pitch_num,triad_num,added_note_num,new_chord_num
5748,10,2,4,2933.0,8.302041,-1,Bbj7,-1,38.0,1.0,...,j7,A#,j7,A#j7,M,j7,0,0,3,49
5756,10,3,4,2937.0,9.519977,-1,Bb-7,-1,45.0,1.0,...,-7,A#,-7,A#-7,-,m7,0,1,1,28
5766,10,4,4,2941.0,10.737528,-1,Eb7,-1,41.0,1.0,...,7,D#,7,D#7,M,m7,4,0,1,8
5781,10,6,4,2949.0,13.080862,-1,Fj7,-1,45.0,1.0,...,j7,F,j7,Fj7,M,j7,3,0,3,13
5787,10,7,4,2953.0,14.247415,-1,Ab-7,-1,49.0,1.0,...,-7,G#,-7,G#-7,-,m7,5,1,1,42
5793,10,8,4,2957.0,15.420181,-1,Db7,-1,38.0,1.0,...,7,C#,7,C#7,M,m7,11,0,1,31
5799,10,9,4,2961.0,16.569388,-1,G-7,-1,47.0,1.0,...,-7,G,-7,G-7,-,m7,1,1,1,5
5811,10,10,4,2965.0,17.753220,-1,C7,-1,40.0,1.0,...,7,C,7,C7,M,m7,2,0,1,12
5820,10,11,4,2969.0,18.940295,-1,A-7,-1,39.0,1.0,...,-7,A,-7,A-7,-,m7,7,1,1,18
5828,10,12,4,2973.0,20.122063,-1,D-7,-1,30.0,1.0,...,-7,D,-7,D-7,-,m7,6,1,1,10
